In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
import json
# from matplotlib import colormaps
import re
import io
from datetime import datetime

from src.metadata_utils import extract_metadata
from src.metadata_utils import DHM_data_extraction
from src.metadata_utils import fatigue_data_extraction
from src.metadata_utils import fatigue_dataframe_extraction
from src.Plot import main_plot
from src.Plot import Plot_single_DHM
from src.Plot import Plot_multi_DHM

#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# This code is used to plot the data from the FE tester for the CV
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

%config InlineBackend.figure_format = 'retina' # For high resolution figures in Jupyter Notebook
#%matplotlib widget

---

## Operator window

#### Parameters to Specify

- **Input file path**  
- **Output file path**  
- **File name**  
- **Number of points to remove** (in case of breakdown)

⚠️ **Important:** Pay attention to the **output path**. The program will automatically create **4 subfolders** in the output path to store the plots and the CSV file.

In [ ]:
# Set the path to the data file
# ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

file_path = r'C:\Users\melan\OneDrive\Documents\Dossier_Nathan\PhD\download\06 - FeCAP-FTJ\02 - Data\02 - FE tester\NASA_FeCAP_07\Endurance measurement - DHM'
output_path = r'C:\Users\melan\OneDrive\Documents\Dossier_Nathan\PhD\download\06 - FeCAP-FTJ\02 - Data\02 - FE tester\NASA_FeCAP_07\Endurance measurement - DHM'
file_name = 'NASA_FeCAP_07_ID_19-09_FM_05.dat'
full_path = os.path.join(file_path, file_name)

# If we have a breakdown during the measurement, we can remove the last n points
point_removal = 0

# Extract the last folder name from the file path
last_folder = os.path.basename(os.path.dirname(file_path))
print(last_folder)

# Séparer nom et extension
base_name = os.path.splitext(file_name)[0]
print(base_name)  # ➜ NASA_SL_ALE_10_ID_095

---

## Complete fatigue analysis

File extraction

In [ ]:
# --- Get the lines where the information of the fatigue measurement are stored ---

data_lines_complete_fatigue = fatigue_data_extraction(full_path)

# --- Initialize metadata placeholders ---

metadata_dict, metadata_str = extract_metadata(data_lines_complete_fatigue, point_removal, keyword="Hysteresis")

# --- Extract the dataframe of the fatigue measurement into a pandas series to plot the curves ---

df_fatigue = fatigue_dataframe_extraction(data_lines_complete_fatigue, point_removal)


Plot Endurance measurement Pr

In [ ]:
# --- Plot the fatigue data ---

label_size = 13
Save_main_plot = True

main_plot(df_fatigue, metadata_dict, metadata_str, output_path, base_name, label_size)

output_path_02 = os.path.join(output_path, base_name)
os.makedirs(output_path_02, exist_ok=True)  # Ensure output directory exists
output_main_plot = os.path.join(output_path_02, "Main plot")
os.makedirs(output_main_plot, exist_ok=True)  # Ensure output directory exists


---

## DHM analysis

Data analysis

In [ ]:

DHM_dataframe = DHM_data_extraction(full_path, Cycles_total, point_removal, df_fatigue)

#Concatenate all DataFrames into a single DataFrame and save to CSV
DHM_dataframe_csv = pd.concat(DHM_dataframe, axis=1)
output_csv_data_path = os.path.join(output_path_02, "CSV data")
os.makedirs(output_csv_data_path, exist_ok=True)  # Ensure output directory exists

DHM_dataframe_csv.to_csv(os.path.join(output_csv_data_path, f"{metadata_dict["Measurement_date_iso"]}_{base_name}_FatigueDHM.csv"), index=False)


Single plot

In [ ]:
# Ploting of every DHM curve into a single plot.

label_size = 13
Plot_single_DHM(DHM_dataframe, Cycles_total, label_size, output_main_plot, base_name, metadata_dict)


Multi-plot

In [ ]:
# Plot of every DHM curve.

label_size = 13
Plot_multi_DHM(DHM_dataframe,output_path_02, Cycles_total, label_size, base_name)
